### Problem Statement

> Online judges provide a platform where many users solve problems everyday to improve their programming skills. The users can be beginners or experts in competitive programming. Some users might be good at solving specific category of problems(e.g. Greedy, Graph algorithms, Dynamic Programming etc.) while others may be beginners in the same. There can be patterns to everything, and the goal of the machine learning would be to identify these patterns and model user’s behaviour from these patterns. The goal of this challenge is to predict range of attempts a user will make to solve a given problem given user and problem details. Finding these patterns can help the programming committee, as it will help them to suggest relevant problems to solve and provide hints automatically on which users can get stuck.  

### Evaluation Metric
The metric used for evaluating the performance of the model is the F1 score between the predicted and the actual value with average parameter set to ‘weighted’.

### Data Files

### train_submissions.csv 
This contains 1,55,295 submissions which are selected randomly from 2,21,850 submissions. Contains 3 columns (‘user_id’, ‘problem_id’, ‘attempts_range’). The variable ‘attempts_range’ denoted the range no. in which attempts the user made to get the solution accepted lies.

|Attempts_range | No. of attempts lies inside|
|---------------|----------------------------|
|1              |                         1-1|
|2              |                         2-3|
|3              |                         4-5|
|4              |                         6-7|
|5              |                         8-9|
|6              |                        >=10|




### user_data.csv - This is the file containing data of users. It contains the following features :-
- user_id - unique ID assigned to each user
- submission_count - total number of user submissions
- problem_solved - total number of accepted user submissions
- contribution - user contribution to the judge
- country - location of user
- follower_count - amount of users who have this user in followers
- last_online_time_seconds - time when user was last seen online
- max_rating - maximum rating of user
- rating - rating of user
- rank - can be one of ‘beginner’ ,’intermediate’ , ‘advanced’, ‘expert’
- registration_time_seconds - time when user was registered

### problem_data.csv - This is the file containing data of the problems. It contains the following features :-
- problem_id - unique ID assigned to each problem
- level_id - the difficulty level of the problem between ‘A’ to ‘N’
- points - amount of points for the problem
- tags - problem tag(s) like greedy, graphs, DFS etc.

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [2]:
if IN_COLAB:
    !pip install xgboost --upgrade
    !pip install yellowbrick
    !pip install mlxtend==0.17.2
    !pip install tpot
    !pip install featuretools

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import featuretools as ft
from mlxtend.preprocessing import TransactionEncoder
from pathlib import Path

In [4]:
### script level variables
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = 'drive/My Drive/datasource/userproblem/{file}'
    source_path = Path.cwd()
else:
    data_path='{file}'
    source_path =Path.cwd().joinpath('datasource')
    
train_file = source_path.joinpath(data_path.format(file='train_submissions.csv'))
test_file = source_path.joinpath(data_path.format(file='test_submissions.csv'))
user_file  = source_path.joinpath(data_path.format(file='user_data.csv'))
problem_file  = source_path.joinpath(data_path.format(file='problem_data.csv'))

In [5]:
def load_data(train_file,test_file,user_file,problem_file):
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    user_df = pd.read_csv(user_file)
    problem_df = pd.read_csv(problem_file)
    return train_df,test_df,user_df,problem_df
    

In [6]:
def extended_describe(dataframe):
    extended_describe_df = dataframe.describe(include='all').T 
    extended_describe_df['null_count']= dataframe.isnull().sum()
    return extended_describe_df

In [7]:
def merge_df(left_df,right_df,how,on,suffixes):
    if not on:
        raise valueError("Unable to join dataframes as join cols not specified")
    how = how or 'left'
    suffixes = suffixes or ('_left','_right')
    return  left_df.merge(right=right_df,how =how,on =on,suffixes=suffixes)
    

In [8]:
train_df,test_df,user_df,problem_df = load_data(train_file,test_file,user_file,problem_file)

In [9]:
# save_df = merge_df(train_df,problem_df,how ='left',on =['problem_id'],suffixes=('_left', '_right'))
# save_df = merge_df(save_df,user_df,how ='left',on =['user_id'],suffixes=('_left', '_right'))
# save_df.to_csv("user_problem.csv")

### Data Exploration

In [10]:
extended_describe(train_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,null_count
user_id,155295,3529,user_1009,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
problem_id,155295,5776,prob_5071,1365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
attempts_range,155295,NaN,NaN,NaN,1.75503,1.07845,1,1,1,2,6,0


In [11]:
extended_describe(test_df)

,count,unique,top,freq,null_count
ID,66555,66555,user_1230_prob_4058,1,0
user_id,66555,3501,user_2744,52,0
problem_id,66555,4716,prob_5071,602,0


In [12]:
extended_describe(user_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,null_count
user_id,3571,3571,user_2324,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
submission_count,3571,NaN,NaN,NaN,299.481,366.103,1,66.5,169,390,4570,0
problem_solved,3571,NaN,NaN,NaN,267.894,344.14,0,53,146,349,4476,0
contribution,3571,NaN,NaN,NaN,4.10249,16.5523,-64,0,0,0,171,0
country,2418,79,India,619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1153
follower_count,3571,NaN,NaN,NaN,46.6906,211.495,0,4,13,40,10575,0
last_online_time_seconds,3571,NaN,NaN,NaN,1.50268e+09,5.11485e+06,1.48424e+09,1.50269e+09,1.50505e+09,1.50555e+09,1.5056e+09,0
max_rating,3571,NaN,NaN,NaN,390.374,92.4288,303.899,317.661,355.791,444.954,983.085,0
rating,3571,NaN,NaN,NaN,350.166,106.593,0,279.243,329.702,413.418,911.124,0
rank,3571,4,beginner,1509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
extended_describe(problem_df)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,null_count
problem_id,6544,6544,prob_2633,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
level_type,6411,14,A,1042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133
points,2627,NaN,NaN,NaN,1452.38,789.542,-1,1000,1500,2000,5000,3917
tags,3060,882,implementation,297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3484


### Few intutions
- user_df country has nulls
- user_df has last_online_time_seconds, registration_time_seconds, we can try to convert them to dates
- problem_df has level_type,points, tags has nulls

In [14]:
def preprocess_user(user_df):
    rank_map = {'beginner':1,'intermediate':2,'advanced':3,'expert':4}
    # Fill country with "Not_specified"
    user_df['registration'] = pd.to_datetime(user_df['registration_time_seconds'], unit='s')
    user_df['last_online'] = pd.to_datetime(user_df['last_online_time_seconds'], unit='s')
    user_df['days_spent'] = (user_df['last_online'] -user_df['registration']).dt.days 
    user_df['rank'] = user_df['rank'].map(rank_map)
    user_df['rating_diff'] = user_df['max_rating']- user_df['rating']
    user_df['rating_ratio'] = user_df['rating']/ user_df['max_rating']
    user_df['country_not_specified'] = user_df['country'].map(lambda x: int(x is np.nan))
    user_df.drop(['country','last_online_time_seconds','registration_time_seconds','registration','last_online'],axis=1,inplace=True)
    return user_df

In [15]:
user_df = preprocess_user(user_df)

In [16]:
user_df.head()

,user_id,submission_count,problem_solved,contribution,follower_count,max_rating,rating,rank,days_spent,rating_diff,rating_ratio,country_not_specified
0,user_3311,47,40,0,4,348.337,330.849,2,433,17.488,0.949796,1
1,user_3028,63,52,0,17,405.677,339.450,2,660,66.227,0.836749,0
2,user_2268,226,203,-8,24,307.339,284.404,1,593,22.935,0.925376,0
3,user_480,611,490,1,94,525.803,471.330,3,1788,54.473,0.896400,0
4,user_650,504,479,12,4,548.739,486.525,3,1169,62.214,0.886624,0


In [17]:
def preprocess_problem(problem_df):
    problem_df['points_&_tags_none'] = problem_df['points'].isna()&problem_df['tags'].isna() 

    problem_df['points_&_tags_none'] = problem_df['points_&_tags_none'].map(lambda x: int(x))

    problem_df.loc[problem_df[(problem_df['level_type']=='A')&(problem_df['points'].isna())].index,'points'] = 500

    problem_df.loc[problem_df[(problem_df['level_type']=='B')&(problem_df['points'].isna())].index,'points'] = 1000

    problem_df.loc[problem_df[(problem_df['level_type']=='C')&(problem_df['points'].isna())].index,'points'] = 1500

    problem_df.loc[problem_df[(problem_df['level_type']=='D')&(problem_df['points'].isna())].index,'points'] = 2000

    problem_df.loc[problem_df[(problem_df['level_type']=='E')&(problem_df['points'].isna())].index,'points'] = 2500

    problem_df.loc[problem_df[(problem_df['level_type']=='F')&(problem_df['points'].isna())].index,'points'] = 2750

    problem_df.loc[problem_df[(problem_df['level_type']=='G')&(problem_df['points'].isna())].index,'points'] = 3000

    problem_df.loc[problem_df[(problem_df['level_type']=='H')&(problem_df['points'].isna())].index,'points'] = 3000

    problem_df.loc[problem_df[(problem_df['points'].isna())].index,'points'] = 2500

    level_type_map = {k:i for i,k in enumerate(sorted(problem_df['level_type'].dropna().unique()))}

    problem_df['level_type'] = problem_df['level_type'].map(lambda x:level_type_map.get(x,-1))

    problem_df['tags'].fillna('',inplace=True)
    return problem_df


def process_tags(problem_df):
    te = TransactionEncoder()
    text_array =problem_df['tags'].str.replace('*','').str.replace('2-','').str.replace('[\s-]','_').str.split(pat=',').values
    te_ary = te.fit(text_array).transform(text_array)
    text_df =pd.DataFrame(te_ary.astype("int"), columns=te.columns_)
    text_df.drop([''],inplace=True,axis=1) 
    problem_df =pd.concat([problem_df, text_df], axis= 1)
    return problem_df

In [18]:
problem_df = preprocess_problem(problem_df)
#problem_df = process_tags(problem_df)

In [19]:
train_df['ID']= train_df['user_id']+'_'+train_df['problem_id']

In [20]:
train_df.head()

,user_id,problem_id,attempts_range,ID
0,user_232,prob_6507,1,user_232_prob_6507
1,user_3568,prob_2994,3,user_3568_prob_2994
2,user_1600,prob_5071,1,user_1600_prob_5071
3,user_2256,prob_703,1,user_2256_prob_703
4,user_2321,prob_356,1,user_2321_prob_356


In [21]:
#full_df = pd.concat([train_df,test_df],ignore_index=True)

In [22]:
full_df = merge_df(train_df,problem_df,how ='left',on =['problem_id'],suffixes=('_left', '_right'))

In [23]:
full_df['attempts_range'].fillna(0,inplace=True)
full_df['attempts_range'].astype(int) 

0         1
1         3
2         1
3         1
4         1
         ..
155290    1
155291    6
155292    1
155293    2
155294    2
Name: attempts_range, Length: 155295, dtype: int32

In [24]:
full_df = merge_df(full_df,user_df,how ='left',on =['user_id'],suffixes=('_left', '_right'))

In [30]:
full_df.to_csv('preprocessed_user_problem.csv',index=False)

In [26]:
# es = ft.EntitySet(id="userproblem_data")

# es = es.entity_from_dataframe(entity_id="user",
#                               dataframe=user_df,
#                               index="user_id",
#                              )

# es = es.entity_from_dataframe(entity_id="full_df",
#                               dataframe=full_df,
#                               index="ID",
#                              )

# new_relationship = ft.Relationship(es["user"]["user_id"],
#                                    es["full_df"]["user_id"])

# es = es.add_relationship(new_relationship)

# es

# feature_matrix_add, feature_names_add = ft.dfs(entityset=es, target_entity = 'full_df', 
#                                        agg_primitives = ['n_most_common', 'num_unique','entropy' ,'median','std'],
#                                        trans_primitives = ['num_words','num_characters'],  
#                                        max_depth = 2, max_features = 100,
#                                        verbose = 1, n_jobs = -1, 
#                                        chunk_size = 100)

# feature_matrix = feature_matrix_add.reindex(index=full_df['ID'])
# feature_matrix = feature_matrix_add.reset_index()

# pd.set_option('display.max_columns', 50)

# feature_matrix.head()

In [27]:
# primitives = ft.list_primitives()
# pd.options.display.max_colwidth = 100
# primitives[primitives['type'] == 'aggregation'] 

In [28]:
#primitives[primitives['type'] == 'transform']

In [29]:
#feature_matrix.to_csv('Preprocessed.csv')